# Задание 5

## 1. ContextTimer(0.3 балла)
Напишите менеджер контекста, который позволит засекать время выполнения блока кода с помощью конструкции with и выводить это время на экран по выходу из блока. Пример использования:

<code>
with Timer ():
    do_some_long_stuff()
</code>

Рекомендации: используйте стандартную библиотеку для работы со временем.

Протестируйте себя.

In [1]:
import time
from math import factorial

In [2]:
class Timer:
    def __enter__(self):
        self.start = time.time()
        
    def __exit__(self, exc_type, exc_value, traceback):
        self.finish = time.time()
        print("Execution took", self.finish - self.start, "seconds")

In [3]:
with Timer():
    print(factorial(20))

2432902008176640000
Execution took 0.0005085468292236328 seconds


## 2.Transaction (0.4 балла)
Вам необходимо написать менеджер контекстов, который позволит безопасно работать с транзакциями. Напишите класс Storage, в котором будут храниться какие-то данные в виде словаря. Эти данные должны быть закрытыми и их можно только читать через операцию []. У этого класса должен быть метод edit, который возвращает менеджер контекста, позволяющий редактировать исходный объект (опять же через []). При этом результаты редактирования записываются в исходный объект только если весь блок выполнился успешно. Пример использования:

<code>
s = Storage()
with s.edit() as se :
    se["a"] = 1
    may_be_an_exception_here()
</code>

Протестируйте себя.

In [4]:
class Storage:
    class ContextManager:
        def __init__(self):
            self.data = dict()
            
        def __enter__(self):
            self.new_data = self.data.copy()
            return self
        
        def __exit__(self, exc_type, exc_value, traceback):
            if exc_type:
                self.new_data = None
                print("Error occured, nothing changed.")
            else:
                self.data = self.new_data
                
        def __getitem__(self, key):
            return self.new_data[key]
        
        def __setitem__(self, key, value):
            self.new_data[key] = value
        
        def __delitem__(self, key):
            del self.new_data[key]
        
    def __init__(self):
        self.context_manager = self.ContextManager()
        
    def edit(self):
        return self.context_manager
    
    def __getitem__(self, key):
        value = self.context_manager.data[key]
        return value
    
    def __str__(self):
        string = "Storage: "
        string += self.context_manager.data.__str__()
        return string

Протестируем себя, разобрав 2 различных сценария: когда происходит исключение и когда все хорошо.

In [5]:
s = Storage()
with s.edit() as se :
    se["a"] = 1
    se["b"] = 42
print(s)
print('Result of s["a"]:', s["a"])
# s["b"] = 50 # it's forbidden
try:
    with s.edit() as se :
        del se["a"]
        se["b"] = 50
        raise Exception()
except:
    pass
print(s)

Storage: {'a': 1, 'b': 42}
Result of s["a"]: 1
Error occured, nothing changed.
Storage: {'a': 1, 'b': 42}


## 3. Phone numbers (0.3)
Напишите регулярное выражение для распознавания телефонных номеров и протестируйте себя. Номера, которые должны распознаваться:
* 3-22-46
* +71239513749
* 71239513749
* 1239513749
* +7(123)-951-37-49
* +7(123)9513749
* +7-123-9513749
* +7-123-951-37-49

* 7(123)-951-37-49
* 7(123)9513749
* 7-123-9513749
* 7-123-951-37-49

* (123)-951-37-49
* (123)9513749
* 123-9513749
* 123-951-37-49

In [6]:
import re

In [7]:
numbers = """
          3-22-46
          +71239513749
          71239513749
          1239513749
          +7(123)-951-37-49
          +7(123)9513749
          +7-123-9513749
          +7-123-951-37-49
          7(123)-951-37-49
          7(123)9513749
          7-123-9513749
          7-123-951-37-49
          (123)-951-37-49
          (123)9513749
          123-9513749
          123-951-37-49
          """.split()
prog = re.compile(r"((\+?7)?.?(\d{3}).{,2}(\d{3}))?.{,2}(\d{2}).?(\d{2})")
result = [prog.match(number).group(0) for number in numbers]
assert(result == numbers)
print(result)

['3-22-46', '+71239513749', '71239513749', '1239513749', '+7(123)-951-37-49', '+7(123)9513749', '+7-123-9513749', '+7-123-951-37-49', '7(123)-951-37-49', '7(123)9513749', '7-123-9513749', '7-123-951-37-49', '(123)-951-37-49', '(123)9513749', '123-9513749', '123-951-37-49']


Как можно заметить, все номера верно определились.